# Classes and Inheritance

- [Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples](#Item-22:-Prefer-Helper-Classes-Over-Bookkeeping-with-Dictionaries-and-Tuples)
- [Item 23: Accept Functions for Simple Interfaces Instead of Classes](#Item-23:-Accept-Functions-for-Simple-Interfaces-Instead-of-Classes)
- [Item 24: Use @classmethod Polymorphism to Construct Objects Generically](#Item-24:-Use-@classmethod-Polymorphism-to-Construct-Objects-Generically)
- [Item 25: Initialize Parent Classes with super](#Item-25:-Initialize-Parent-Classes-with-super)
- [Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes](#Item-26:-Use-Multiple-Inheritance-Only-for-Mix-in-Utility-Classes)
- [Item 27: Prefer Public Attributes Over Private Ones](#Item-27:-Prefer-Public-Attributes-Over-Private-Ones)

## Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [ ]:
import collections

Grade = collections.namedtuple('Grade', ('score', 'weight'))

class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight
    

class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    

class Gradebook(object):
    def __init__(self):
        self._students = {}
    
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]
    
    
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(90, 0.1)
math.report_grade(100, 0.9)
print(albert.average_grade())

- Avoid making dictionaries with values that are other dictionaries or long tuples.
- Use `collections.namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.

## Item 23: Accept Functions for Simple Interfaces Instead of Classes

Many of Python's built-in APIs allow you to customize behavior by passing in a function. These hooks are used by APIs to call back your code while they execute

In [ ]:
names = ['Apollo', 'Muse', 'Zeus', 'Venus']
names.sort(key=lambda x: len(x))
print(names)

In [ ]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count


result, count = increment_with_report(current, increments)
assert count == 2


# class way
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0
    
counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2


class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0
    
    
counter = BetterCountMissing()
counter()
assert callable(counter)

result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2



## Item 24: Use *@classmethod* Polymorphism to Construct Objects Generically

In [ ]:
class InputData(object):
    def read(self):
        raise NotImplementedError


class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()


class Worker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError


class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result
        

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))
        

def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result


def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

In [ ]:
class GenericInputData(object):
    def read(self):
        raise NotImplementedError
        
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError
        

class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))
            

class GenericWorker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError
        
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers


class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result
        

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result


def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)


Now you can write other *GenericInputData* and *GenericWorker* classes as you wish and not have to rewrite any of the glue code.

## Item 25: Initialize Parent Classes with *super*

In [ ]:
# old way
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value


class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

### Diamond inheritance

**Diamond inheritance**. Diamond inheritance happens when a subclass inherits from two separate classes that have the same super class somewhere in the hierarchy. Diamond inheritance causes the common superlass's \_\_init__ method to run multiple times, causing unexpected behavior.

In [ ]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        print('init MyBaseClass')


class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        

class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2
        
        
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        

foo = ThisWay(5)
# the __init__ of MyBaseClass will be called twice
print('Should be (5 * 5) + 2 = 27 but is ', foo.value)

To solve these problems, Python 2.2 added the *super* built-in function and defined the method resolution order (MRO). The MRO standardizes which superclasses are initialized before other (e.g., depth-first, left-to-right). It also ensure that common super classes in diamond hierarchies are only run once.

In [ ]:
# Python 3
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
        
class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2
        

class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)
        

foo = GoodWay(5)
print('Should be 5 * (5 + 2) = 35 and is', foo.value)

from pprint import pprint
pprint(GoodWay.mro())

The *super* built-in function works well, but it still has two noticeable problems in Python2:

- Its syntax is a bit verbose. You have to specify the class you're in, the self object, the method name (usually \_\_init__), and all the arguments. This construction can be confusing to new Python programmers.
- You have to specify the current class by name in the call to *super*. If you ever change the class's name -- a very common activity when improving a class hierarchy -- you also need to update every call to *super*

In [ ]:
# Python 3
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        print('init MyBaseClass')


class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)


class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        

assert Explicit(10).value == Implicit(10).value

## Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes

It's better to avoid multiple inheritance altogether.

If you find yourself desiring the convenience and encapsulation that comes with multiple inheritance, consider writing a mix-in instead. A mix-in is a small class that only defines a set of additional methods that a class should provide. Mix-in classes don't define their own instance attributes nor require their \_\_init__ constructor to be called.

Mix-ins can be composed and layered to minimize repetitive code maximize reuse.


In [ ]:
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value
        
    
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left, self.right = left, right
        

tree = BinaryTree(10,
        left=BinaryTree(7, right=BinaryTree(9)),
        right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())


class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None,
                 right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
        
    # Override default behavior
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and
               key == 'parent'):
            return value.value  # Prevent cycles
        else:
            return super()._traverse(key, value)
        
        
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())


class JsonMixin(object):
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

### Things to Remember

- Avoid using multiple inheritance if mix-in classes can achieve the same outcome.
- Use pluggable behaviors at the instance level to provide per-class customization when mix-in classes may require it.
- Compose mix-ins to create complex functionality from simple behaviors.

## Item 27: Prefer Public Attributes Over Private Ones

In Python, there are only two types of attribute visibility for a class's attributes: *public* and *private*.


In [ ]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field
    

'''
Directly accessing private fields from outside
the class raises an exception
'''
foo = MyObject()
# foo.__private_field  # Will raise exception


class MyParentObject(object):
    def __init__(self):
        self.public_field = 55
        self.__private_field = 71
     
    
class MyChildObject(MyParentObject):
    def get_parent_private_field(self):
        return self.__private_field

'''
A subclass can't access its parent class's
private fields.
'''
bar = MyChildObject()
# bar.get_parent_private_field()
print(bar.__dict__)
# access without permission
assert bar._MyParentObject__private_field == 71

Why doesn't the syntax for private attributes actually enforce strict visibility? The simplest answer is one often-quoted motto of Python: **"We are all consenting adults here".** Python programmers believe that the benefits of being open outweigh the downsides of being closed.

### Things to Remember

- Private attributes aren't rigorously enforced by the Python compiler
- Plan from the beginning to allow subclasses to do more with your internal APIs and attributes instead of locking them out by default.
- Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.
- Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control.

## Item 28: Inherit from *collections.abs* for Custom Container Types

In [ ]:
class FrequencyList(list):  # subclass of list
    def __init__(self, members):
        super().__init__(members)
        
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts
    

foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'c', 'a'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

In [ ]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

### Things to Remember

- Inherit directly from Python's container types (like *list* or *dict*) for simple use cases.
- Beware of the large number of methods required to implement custom container types correctly.
- Have your custom container types inherit from the interfaces defined in **collections.abc** to ensure that your classes match required interfaces and behaviors